In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt


#tf.compat.v1.enable_eager_execution()
#tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

# Weights

In [23]:
weights = {
    # 5x5 conv, 1 input, 6 outputs
    'weights_conv_1': tf.Variable(tf.random.normal([5, 5, 1, 6])),
    # 5x5 conv, 6 inputs, 16 outputs
    'weights_conv_2': tf.Variable(tf.random.normal([5, 5, 6, 16])),
    #5x5 conv as in paper, 16 inputs, 120 outputs
    'weights_conv_3': tf.Variable(tf.random.normal([5, 5, 16, 120])),
    # fully connected, 5*5*16 inputs, 120 outputs
    'weights_dense_1': tf.Variable(tf.random.normal([5*5*16, 120])),
    # fully connected, 120 inputs, 84 outputs
    'weights_dense_2': tf.Variable(tf.random.normal([120, 84])),
    # 84 inputs, 10 outputs (class prediction)
    'weights_dense_3': tf.Variable(tf.random.normal([84, 10])),
}

masks = {
    # 5x5 conv, 1 input, 6 outputs
    'mask_conv_1': tf.Variable(tf.ones([5, 5, 1, 6]), trainable=False),
    # 5x5 conv, 6 inputs, 16 outputs
    'mask_conv_2': tf.Variable(tf.ones([5, 5, 6, 16]), trainable=False),
    #5x5 conv as in paper, 16 inputs, 120 outputs
    'mask_conv_3': tf.Variable(tf.ones([5, 5, 16, 120]), trainable=False),
    # fully connected, 5*5*16 inputs, 120 outputs
    'mask_dense_1': tf.Variable(tf.ones([5*5*16, 120])),
    # fully connected, 120 inputs, 84 outputs
    'mask_dense_2': tf.Variable(tf.ones([120, 84])),
    # 84 inputs, 10 outputs (class prediction)
    'mask_dense_3': tf.Variable(tf.ones([84, 10])),
}

biases = {
    #output depth
    'bias_conv_1': tf.Variable(tf.random.normal([6])),
    'bias_conv_2': tf.Variable(tf.random.normal([16])),
    'bias_dense_1': tf.Variable(tf.random.normal([120])),
    'bias_dense_2': tf.Variable(tf.random.normal([84])),
    'bias_dense_3': tf.Variable(tf.random.normal([10])),
}



In [24]:
def weight_mask_variable(var, scope):

    mask = tf.Variable(initial_value = tf.ones(var.shape),
                       trainable=False
                      )
    return mask

def apply_mask(x, scope=''):
    #print(x.shape)
    mask = weight_mask_variable(x, scope)
    #threshold = weight_threshold_variable(x, scope)

    x = tf.cast(x, tf.float32)
    masked_weights = tf.multiply(mask, x)
    

    return masked_weights

# Wrappers

In [55]:
#conv2D with bias and relu activation

class CustomConvLayer(layers.Layer):

    def __init__(self, weights, mask, biases, strides, padding='SAME'):
        
        super(CustomConvLayer, self).__init__()
        self.w = weights
        self.m = mask
        self.b = biases
        self.s = strides
        self.p = padding

        
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p)
        
        x = tf.nn.bias_add(x, self.b)
        return tf.nn.tanh(x)
        

#Average Pooling Layer
class CustomPoolLayer(layers.Layer):
    
    def __init__(self, k=2, padding='VALID'):
        super(CustomPoolLayer, self).__init__()
        self.k = k
        self.p = padding
    
    def call(self, inputs):
        return tf.nn.avg_pool2d(inputs, ksize=[1, self.k, self.k,1], strides=[1, self.k, self.k, 1], padding=self.p)

    
#Dense Layer with Bias
class CustomDenseLayer(layers.Layer):
    
    def __init__(self, weights, bias, activation = 'tanh'):
        super(CustomDenseLayer, self).__init__()
        self.w = weights
        self.b = bias
        self.a = activation
        
    def call(self, inputs):
        #print(self.w)
        #print(inputs)
        x = tf.matmul(inputs, self.w)
        #print(x)
        x = tf.nn.bias_add(x, self.b)
        if self.a == 'tanh':
            return tf.nn.tanh(x)
        if self.a == 'softmax':
            return tf.nn.softmax(x)


# Create Model

In [56]:
class CustomConvModel(tf.keras.Model):
    def __init__(self):
        super(CustomConvModel, self).__init__()
        self.conv1 = CustomConvLayer(weights['weights_conv_1'], masks['mask_conv_1'], biases['bias_conv_1'], 1, 'SAME')#'VALID')
        self.maxpool1 = CustomPoolLayer(k=2, padding='SAME')
        self.conv2 = CustomConvLayer(weights['weights_conv_2'], masks['mask_conv_2'], biases['bias_conv_2'], 1, 'VALID')
        self.maxpool2 = CustomPoolLayer(k=2, padding='VALID')
        self.conv3 = CustomConvLayer(weights['weights_conv_3'], masks['mask_conv_3'], biases['bias_dense_1'], 1, 'VALID')
        #self.dense1 = CustomDenseLayer(weights['weights_dense_1'], biases['bias_dense_1'], 'tanh')
        self.dense2 = CustomDenseLayer(weights['weights_dense_2'], biases['bias_dense_2'], 'tanh')
        self.dense3 = CustomDenseLayer(weights['weights_dense_3'], biases['bias_dense_3'], 'softmax')
        
    def call(self, inputs):
        x = tf.reshape(inputs, shape=[-1, 28, 28, 1])
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.maxpool1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.maxpool2(x)
        #print('yo',x.shape)
        #x = layers.Flatten()(x)
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)
        x = layers.Flatten()(x)
        print(x.shape)
        x =  self.dense2(x)
        print(x.shape)
        x =  self.dense3(x)
        print(x.shape)
        return x
        

In [57]:
model = CustomConvModel()

In [58]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
             )

In [59]:
model.fit(x=x_train[:30000],
          y=y_train[:30000],
          batch_size=64,
          epochs=2,
          validation_data=(x_test, y_test),
         )

(None, 28, 28, 1)
(None, 28, 28, 6)
(None, 14, 14, 6)
(None, 10, 10, 16)
(None, 5, 5, 16)
(None, 1, 1, 120)
(None, 120)
(None, 84)
(None, 10)
Train on 30000 samples, validate on 10000 samples
Epoch 1/2
30000/30000 [==============================] - 39s 1ms/sample - loss: 1.5956 - accuracy: 0.8667 - val_loss: 1.5886 - val_accuracy: 0.8732
Epoch 2/2
30000/30000 [==============================] - 35s 1ms/sample - loss: 1.5906 - accuracy: 0.8715 - val_loss: 1.5876 - val_accuracy: 0.8749


In [60]:
len(model.get_weights())

13

In [64]:
for layer in model.get_weights():
    print(layer.shape)



(5, 5, 1, 6)
(6,)
(5, 5, 1, 6)
(5, 5, 6, 16)
(16,)
(5, 5, 6, 16)
(5, 5, 16, 120)
(120,)
(5, 5, 16, 120)
(120, 84)
(84,)
(84, 10)
(10,)


In [46]:
for layer in model.get_weights():
    print(layer.shape)



(5, 5, 1, 6)
(6,)
(5, 5, 1, 6)
(5, 5, 6, 16)
(16,)
(5, 5, 6, 16)
(5, 5, 16, 120)
(120,)
(5, 5, 16, 120)
(120, 84)
(84,)
(84, 10)
(10,)


In [73]:
model.get_weights()[0][0]

array([[[ 1.4772917 , -0.96927375,  1.0066881 ,  2.0744536 ,
         -0.70605135,  1.3498781 ]],

       [[-0.6492623 ,  1.1915262 ,  0.7286109 , -1.6436797 ,
         -0.21559437,  0.06576186]],

       [[-0.91879785, -0.564915  ,  0.03131934,  1.2240024 ,
          0.8457885 ,  0.522353  ]],

       [[ 0.9856671 , -0.5873115 ,  0.02142585,  1.065783  ,
         -0.61659753, -0.8304464 ]],

       [[-0.2127563 , -1.1773119 ,  0.35823378,  0.16824959,
          1.0378606 ,  0.52482694]]], dtype=float32)

In [ ]:
keras_model = tf.keras.Sequential()
keras_model.add(layers.Reshape((28, 28,1)))
#differences convolutional layer: 
#keras.layers.Conv2D uses 'glorot_uniform' initializer
#keras.layers.Conv2D uses padding method 'valid' as default
keras_model.add(layers.Conv2D(filters=6, kernel_size=(5,5), activation='tanh', padding='SAME',input_shape=(28,28,1)))
#differences pooling layer: 
#keras.layers.AvgPooling2D uses padding method 'valid' (vs. 'SAME' in custom) as default
keras_model.add(layers.AvgPool2D(padding='SAME'))
keras_model.add(layers.Conv2D(filters=16, kernel_size=(5,5), activation='tanh'))
keras_model.add(layers.AvgPool2D())
keras_model.add(layers.Flatten())
keras_model.add(layers.Dense(120, activation='tanh'))
keras_model.add(layers.Dense(84, activation='tanh'))
keras_model.add(layers.Dense(10, activation='softmax'))

In [ ]:
keras_model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                    metrics=['accuracy'],
                    experimental_run_tf_function=False
                   )

In [ ]:
keras_model.fit(x=x_train,
                y=y_train,
                batch_size=64,
                epochs=2,
                validation_data=(x_test, y_test),
         )

In [ ]:
keras_model.summary()

In [ ]:
preds = map (lambda pred: np.argmax(pred), model.predict(x_test[:30]))
preds = list(preds)

In [ ]:


plt.figure(figsize=(10,10))
for i in range(25):
    img = tf.reshape(x_test[i], shape=[28, 28])
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img)

    plt.xlabel(preds[i])
plt.show()